<a href="https://colab.research.google.com/github/Kaleab1999/Ham-Spam-classifier/blob/main/word2vec_with_pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

loading the dataset
separate the dataset by 't' or tab
label the separatted data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df=pd.read_csv('/content/drive/MyDrive/dataset/ssms_spam.txt',sep='\t',
                           names=["label", "message"])
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

## data cleaning

In [7]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
import gensim.downloader as api
import numpy as np
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import spacy
import string

In [10]:
wv = api.load('word2vec-google-news-300')

[=================================================-] 99.8% 1660.2/1662.8MB downloaded


In [5]:
def sent_vec(sent):
    vector_size = wv.vector_size
    wv_res = np.zeros(vector_size)
    # print(wv_res)
    ctr = 1
    for w in sent:
        if w in wv:
            ctr += 1
            wv_res += wv[w]
    wv_res = wv_res/ctr
    return wv_res

In [12]:
# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    doc = nlp(sentence)



    # print(doc)
    # print(type(doc))

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() for word in doc ]

    # print(mytokens)

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [16]:
nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words
print(stop_words)

punctuations = string.punctuation
print(punctuations)

{'thru', 'own', 'nevertheless', 'our', 'cannot', 'whoever', 'thereby', 'beforehand', 'between', 'third', 'front', 'my', 'they', 'else', 'for', 'serious', 'until', 'various', 'eight', 'had', 'enough', 'hers', 'per', 'during', 'otherwise', 'it', 'part', 'less', 'former', 'about', 'same', 'ours', 'latterly', '‘re', 'her', 'fifteen', 'what', 'without', 'nor', 'would', '‘ve', 'anyone', 'besides', 'go', 'nobody', 'or', 'amount', '‘d', 'though', 'hereby', 'when', 'nothing', 'within', 'both', 'yet', 'whole', 'down', 'yourselves', 'nine', 'move', 'how', 'this', 'one', 'much', 'twelve', 'are', 'ever', 'every', 'still', 'am', '‘ll', 're', 'unless', 'all', 'eleven', 'put', 'side', 'herself', 'hence', 'thence', 'upon', 'meanwhile', 'there', 'due', 'onto', 'than', 'against', 'whether', 'last', 'well', 'anyhow', 'sometime', 'at', 'seems', 'however', 'from', "'ll", 'thus', 'must', 'anyway', '’ll', 'name', 'moreover', 'done', 'anything', 'i', 'an', 'take', 'somehow', 'several', 'you', 'top', 'those', '

In [17]:
df['tokens'] = df['message'].apply(spacy_tokenizer)
df.head()

,label,message,tokens
0,ham,"Go until jurong point, crazy.. Available only ...","[jurong, point, crazy, .., available, bugis, n..."
1,ham,Ok lar... Joking wif u oni...,"[ok, lar, ..., joke, wif, u, oni, ...]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,ham,U dun say so early hor... U c already then say...,"[u, dun, early, hor, ..., u, c, ...]"
4,ham,"Nah I don't think he goes to usf, he lives aro...","[nah, think, usf, live]"


In [18]:
df['vector']= df['tokens'].apply(sent_vec)
df.head()

,label,message,tokens,vector
0,ham,"Go until jurong point, crazy.. Available only ...","[jurong, point, crazy, .., available, bugis, n...","[-0.024071326622596152, 0.04204852764423077, 0..."
1,ham,Ok lar... Joking wif u oni...,"[ok, lar, ..., joke, wif, u, oni, ...]","[-0.07352338518415179, 0.0494384765625, 0.0382..."
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, 2, wkly, comp, win, fa, cup, fin...","[-0.002729976878446691, -0.03127872242647059, ..."
3,ham,U dun say so early hor... U c already then say...,"[u, dun, early, hor, ..., u, c, ...]","[-0.07310267857142858, 0.06766183035714286, 0...."
4,ham,"Nah I don't think he goes to usf, he lives aro...","[nah, think, usf, live]","[-0.0390380859375, 0.031988525390625, 0.045922..."


In [27]:
y=pd.get_dummies(df['label'])
y=y.iloc[:,1].values

In [28]:
y=y.tolist()
X = df['vector'].to_list()

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y)

In [30]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

In [31]:
classifier.fit(X_train,y_train)

LogisticRegression()

In [32]:
from sklearn import metrics
predicted = classifier.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.9641255605381166
Logistic Regression Precision: 0.9224806201550387
Logistic Regression Recall: 0.7986577181208053


In [33]:
from sklearn.ensemble import RandomForestClassifier

randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy')
randomclassifier.fit(X_train,y_train)

RandomForestClassifier(criterion='entropy', n_estimators=200)

In [34]:
## Predict for the Test Dataset

predictions = randomclassifier.predict(X_test)

In [38]:
## Import library to check accuracy
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

matrix=confusion_matrix(y_test,predictions)
print(matrix)
score=accuracy_score(y_test,predictions)
print(score)
report=classification_report(y_test,predictions)
print(report)

[[966   0]
 [ 36 113]]
0.967713004484305
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       966
           1       1.00      0.76      0.86       149

    accuracy                           0.97      1115
   macro avg       0.98      0.88      0.92      1115
weighted avg       0.97      0.97      0.97      1115



In [39]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)

pred= gnb.predict(X_test)

from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

matrix=confusion_matrix(y_test,pred)
print(matrix)
score=accuracy_score(y_test,pred)
print(score)
report=classification_report(y_test,pred)
print(report)

[[921  45]
 [ 20 129]]
0.9417040358744395
              precision    recall  f1-score   support

           0       0.98      0.95      0.97       966
           1       0.74      0.87      0.80       149

    accuracy                           0.94      1115
   macro avg       0.86      0.91      0.88      1115
weighted avg       0.95      0.94      0.94      1115

